<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Знакомство-с-данными" data-toc-modified-id="Знакомство-с-данными-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Знакомство с данными</a></span></li><li><span><a href="#Предобработка-текста" data-toc-modified-id="Предобработка-текста-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Предобработка текста</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

### Знакомство с данными

In [18]:
#импорт всех библиотек
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import nltk
from nltk.corpus import stopwords as nltk_stopwords
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score


<div class="alert alert-block alert-success">
<b>Успех:</b> Импорты на месте
</div>  



In [2]:
data = pd.read_csv('/datasets/toxic_comments.csv')

In [3]:
data.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


**С данными успешно познакомились. 159291 строк и 3 столбца.**

### Предобработка текста

In [5]:
#зададим параметры для spacy для лемматизации текста, а также загрузим стоп-слова для английского
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
nltk.download('stopwords')
stopwords = nltk_stopwords.words('english')

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [6]:
#создадим различные функции для обработки текста
def words_only(text):
    return ' '.join(re.sub(r'[^a-zA-Z]', ' ', text.lower()).split())

def lemmatize_text(text, nlp):
    return [token.lemma_ for token in nlp(text)]

def remove_stopwords(text, stopwords):
    return [word for word in text if not word in stopwords and len(word) > 1]

def clean_text(text, nlp, stopwords):

    # очистка от символов
    text_words = words_only(text)

    # лемматизация
    text_lemmas = lemmatize_text(text_words, nlp)

    # очистка от стоп-слов
    return ' '.join(remove_stopwords(text_lemmas, stopwords))

In [7]:
#применим нашу предобработку к тексту
data['text'] = data['text'].apply(lambda x: clean_text(x, nlp, stopwords))

**Успешно провели предобработку текста, используя функции. Использовали лемматизацию, стоп-слова.**

## Обучение

In [51]:
RANDOM_STATE = 42
#разобъем данные
X_train, X_test, y_train, y_test = train_test_split(data['text'], data['toxic'], test_size=0.15, random_state=RANDOM_STATE)

In [52]:
#Преобразуем текст в матрицу
vectorizer = TfidfVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

In [53]:
model = LogisticRegression(random_state=RANDOM_STATE, max_iter=1000)

In [54]:
model.fit(X_train_vec, y_train)

LogisticRegression(max_iter=1000, random_state=42)

In [55]:
y_pred = model.predict(X_test_vec)

In [56]:
score = f1_score(y_test, y_pred)
print(score)

0.7547636723583272


**Достигнуто нужное число метрики**

## Выводы

**Успешно загрузили и подготовили данные, обучили модель логистической регрессии, получили подходящее значение метрики F1.**

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Данные загружены и подготовлены
- [x]  Модели обучены
- [x]  Значение метрики *F1* не меньше 0.75
- [x]  Выводы написаны